# APAN5400: Managing Data 
### Assignment 5 - NoSQL Search Engine: Elasticsearch

### Chowdhury Walid Tahabee
#### 02/23/2021

#### Installing Elasticsearch loader to import CSV and JSON files

In [1]:
!pip3 install elasticsearch-loader

##### Importing JSON object file to Elasticsearch index

In [2]:
!elasticsearch_loader --index companies json companies.json

{'index': 'companies', 'bulk_size': 500, 'es_host': ('http://localhost:9200',), 'verify_certs': False, 'use_ssl': False, 'ca_certs': None, 'http_auth': None, 'delete': False, 'update': False, 'progress': False, 'type': '_doc', 'id_field': None, 'as_child': False, 'with_retry': False, 'index_settings_file': None, 'timeout': 10.0, 'encoding': 'utf-8', 'keys': [], 'es_conn': <Elasticsearch([{'host': 'localhost', 'port': 9200}])>}
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


#### Installing elasticsearch Python packages

In [3]:
!pip3 install elasticsearch
!pip3 install elasticsearch-dsl

#### Connecting to Elasticsearch on my machine

##### 1: Using elasticsearch package

In [4]:
from elasticsearch import Elasticsearch
client = Elasticsearch()

##### 2. Using elasticsearch-dsl package

In [5]:
from elasticsearch_dsl import connections
client = connections.create_connection()

# 1. Which company is the oldest?

In [178]:
#Search the companies index, find where description field contains the word "American" 
# and sort by number of employees in descending order.
# and Return all records.
# print only only the company names and number of employees meeting this criteria
s = Search(using=client, index="companies") \
         .sort({'founded': {'order': 'asc'}})

response = s[:1].execute() # by default you get 10 hits

for hit in response:
    print("The oldest company is "+str(hit.company)+". The company was founded in "+str(hit.founded))

The oldest company is Daimler AG. The company was founded in 1926


In this query, I sorted the 'founded' key in ascending order and set the scroll size to 1 to only print the oldest company.

# 2. Which company's product is a search engine?

In [176]:
s = Search(using=client, index="companies") \
         .query("match", description="search engine") \

response = s[:1].execute()

for hit in response: 
    print("The company that produces search engines is "+str(hit.company))

The company that produces search engines is Google


In this query, I specified the string type key 'description' to be string matched with 'search engine' to print the company that produces search engines. Similarly to question 1, I set the scroll length to 1 to avoid repetition in the printed statement.

# 3. Which companies are headquartered in the USA?

In [162]:
s = Search(using=client, index="companies") \
         .query("match", country="USA") \

response = s[:4].execute()

for hit in response:
    print("The Company Name is "+str(hit.company)+". The company is headquartered in "+str(hit.city)+", "+str(hit.state)+", " +str(hit.country))

The Company Name is Apple. The company is headquartered in Cupertino, California, USA
The Company Name is Microsoft. The company is headquartered in Redmond, Washington, USA
The Company Name is Amazon. The company is headquartered in Seattle, Washington, USA
The Company Name is Google. The company is headquartered in Mountain View, California, USA


In this query, I have specified the string type country key to match with the string 'USA' to capture all companies that are headquartered in the U.S. Similarly to questions 1 and 2 - after running the query initially, I noticed that there are only 4 unique hits, so I subsequently set the scroll size to 4 to only print the unique hits.

# 4. Which companies have been founded in 1990s?

In [174]:
s = Search(using=client, index="companies") \
         .filter('range', founded = {'gte': 1990, 'lte': 1999}) \

response = s[:3].execute()

for hit in response:
    print("The Company Name is "+str(hit.company)+". The company was founded in "+str(hit.founded))

The Company Name is Amazon. The company was founded in 1994
The Company Name is Google. The company was founded in 1998
The Company Name is Alibaba. The company was founded in 1999


In this query, I have specified the range of the 'founded' key to be greater than equal to 1990 and less than equal to 1999 to capture all the companies that were founded in the 90s. Similarly to previous questions - after initially running the query I realized there are only 3 unique hits, so I set the scroll size to 3.  

# 5. What is the total annual revenue of the three companies in Asia?

In [205]:
from elasticsearch_dsl import Q
s = Search(using=client, index="companies")
s.query = Q('bool', must_not=[Q('match', country='USA'), Q('match', country='Germany')])
response = s[:3].execute()

total_revenue = 0
for hit in response:
    total_revenue+=hit.annual_revenue
    
print('Total revenue for asian companies is $' + str(total_revenue) + ' Million')

Total revenue for asian companies is $358 Million


In this query, I searched the company index to find all records that excludes 'USA' and 'Germany' for the 'country' key. After initially running the query in the for loop, I noticed that it only resulted in 3 unique hits, so I adjusted the scroll size accordingly. Subsequently, I created a total_revenue variable and set it to zero before adding all the hits to it using the for loop. 